In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import folium # plotting library
from folium import plugins

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

import seaborn as sns
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

## 1. Question and problem definition

จากปัญหาสภาพอากาศของสหรัฐอเมริกาที่มีความแตกต่าง ที่ส่งผลต่อการลงจอดของเครื่องบิน ดั้งนั้นเราจึงอยากจะวิเคราะห์สภาพอากาศที่แตกต่างของสนามบินนั้นๆด้วยเพื่อให้เห็นภาพข้อมูลของสภาพอากาศ และในการเลือกลงจอดของเครื่องบินโดยวิธีของUnsupervised Learningแบบclustering

## 2. Acquire training and testing data



In [ ]:
#ดึงข้อมูลในpd
df = pd.read_csv('../input/us-weather-events/US_WeatherEvents_2016-2019.csv') 

df.head()

## 3. Wrangle, prepare, cleanse the data

In [ ]:
#ตรวจสอบรูปแบบของข้อมูล
df.info()

In [ ]:
#เช็คดูข้อมูลสภาพอากาศ
df['Type'].value_counts()


In [ ]:
#เช็คดูข้อมูลสภาพอากาศ
df['Severity'].value_counts()

clean data โดยการกรองข้อมูลของ Severity ที่เป็น UNK(unknown) และ Other ตัดทิ้งไปเนื่องจากไม่จำเป็นต้องใช้

In [ ]:

df = df[(df['Severity'] != 'UNK') & (df['Severity'] != 'Other')]

df.head()

In [ ]:
#เช็คดูเหตุการณ์สนามบินกับประเภทของสภาพอากาศที่เกิดขึ้น
df_types = df[['AirportCode','Type']]

df_types.head()

In [ ]:
#เช็คจำนานเหตุการณ์สภาพอากาศของแต่ละสนามบิน
types = pd.get_dummies(df_types['Type']) 

types['AirportCode'] = df_types['AirportCode']

types = types.groupby('AirportCode').sum().reset_index()

types.head()

## 4. Analyze, identify patterns, and explore the data.
 k-Means Clustering


In [ ]:
codes = types[['AirportCode']]
types.drop('AirportCode', axis=1, inplace=True)

การคำนวณหาจุดของเส้นโค้งที่มีจำนวน Cluster ที่เหมาะสมที่สุดนั้นโดยการเลือกค่า k ที่เหมาะสมหรือการหาค่า Optimal cluster number จะใช้วิธี Elbow method

In [ ]:
#Elbow method 
distortions = [] #สร้างอาเรย์ไว้เก็บค่าความคลาดเคลื่อน
K = range(1,20)
for k in K: #วนลูปเพื่อหาค่าKmean
    kmean = KMeans(n_clusters=k, random_state=0, n_init = 50, max_iter = 500)
    kmean.fit(types) #คำสั่งเพื่อจำลองการเรียนรู้ของตัวแปร types
    distortions.append(kmean.inertia_) #.inertia_คือค่าผลรวมความคลาดเคลื่อนกำลังสอง (SSE) ของการแบ่งกลุ่ม

In [ ]:
#พล็อตกราฟ
plt.figure(figsize=(10,5))
plt.plot(K, distortions, 'bx-')
plt.xlabel('k')
plt.ylabel('Distortion')
plt.title('The Elbow Method')
plt.show()

จากกราฟ The elbow method ทำให้ได้ค่า Optimal cluster number อยู่ที่ค่า 4 หรือ ค่า 5 ซึ่งได้มาจากการลากเส้นตรงจากจุดเริ่มไปยังปลายเส้นโค้ง จากนั้นหาระยะจากเส้นตรงตั้งฉากกับเส้นโค้งที่มีระยะห่างมากที่สุด

In [ ]:
# run k-means clustering โดยใช้ค่า 4 
kmeans = KMeans(n_clusters=4, random_state=0).fit(types)

codes['cluster'] = kmeans.labels_
codes.head()

หา PCA ซึ่งคือ PCA คือการหาตัวแปรที่สำคัญเมื่ออ้างอิงจากข้อมูล

In [ ]:
#เตรียมบีบอัดข้อมูล ตัดตัวแปรที่มีความสำคัญน้อยออกไป
pca = PCA().fit(types)
pca_types = pca.transform(types)
print("Variance explained by each component (%): ")
for i in range(len(pca.explained_variance_ratio_)):
      print("\n",i+1,"º:", pca.explained_variance_ratio_[i]*100)
print("Total sum (%): ",sum(pca.explained_variance_ratio_)*100)
print("Explained variance of the first two components (%): ",sum(pca.explained_variance_ratio_[0:1])*100)

เนื่องจากจำนวนตัวอย่างมีขนาดใหญ่กว่าจำนวนคุณสมบัติเราจึงแก้ปัญหาจาก 5 PC ให้เลือกแค่สอง PC แรกซึ่งจะเห็นได้ว่าก็ยังสามารถรักษาข้อมูลได้ถึง 63% ดังนั้นถึงลดขนาดข้อมูลของเรามาใช้สอง PC แรกเพื่อให้เห็นภาพง่ายขึ้น


## 5. Model, predict and solve the problem.

พล็อตกราฟค่าที่ได้จากการทำ PCA เพื่อสร้าง Model 

In [ ]:
c0 = [] #สร้างอาร์เรย์ไว้เก็บค่า pca_types 
c1 = []
c2 = []
c3 = []
 
for i in range(len(pca_types)):  #วนลูปเพื่อเช็คค่า kmeans.labels_[i]จากนั้น append ค่า pca_types เก็บไว้ในอาร์เรย์
    if kmeans.labels_[i] == 0:
        c0.append(pca_types[i])
    if kmeans.labels_[i] == 1:
        c1.append(pca_types[i])
    if kmeans.labels_[i] == 2:
        c2.append(pca_types[i])
    if kmeans.labels_[i] == 3:
        c3.append(pca_types[i])
        
        
c0 = np.array(c0) #สร้างตัวแปรเก็บค่าอาร์เรย์
c1 = np.array(c1)
c2 = np.array(c2)
c3 = np.array(c3)

plt.figure(figsize=(7,7)) #เริ่มพล็อตกราฟ
plt.scatter(c0[:,0], c0[:,1], c='red', label='Cluster 0')
plt.scatter(c1[:,0], c1[:,1], c='blue', label='Cluster 1')
plt.scatter(c2[:,0], c2[:,1], c='green', label='Cluster 2')
plt.scatter(c3[:,0], c3[:,1], c='black', label='Cluster 3')
plt.legend()
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.title('Low dimensional visualization (PCA) - Airports');

In [ ]:
#แสดงตารางข้อมูลสภาพอากาศ  cluster และ สนามบิน
types['cluster']  = kmeans.labels_

types.head()

In [ ]:
types.groupby('cluster').mean() #แสดงค่าเฉลี่ยของสภาพอากาศในแต่ละ cluster

พล็อตกราฟของค่า cluster ในแต่ละสภาพอากาศ

In [ ]:
#สภาพอากาศ Cold
sns.catplot(x='cluster', y='Cold', data=types, kind='bar');

In [ ]:
#สภาพอากาศ Fog
sns.catplot(x='cluster', y='Fog', data=types, kind='bar');

In [ ]:
#สภาพอากาศ Rain
sns.catplot(x='cluster', y='Rain', data=types, kind='bar');

In [ ]:
#สภาพอากาศ Snow
sns.catplot(x='cluster', y='Snow', data=types, kind='bar');

In [ ]:
#สภาพอากาศ Storm
sns.catplot(x='cluster', y='Storm', data=types, kind='bar');

จากการพล็อตกราฟ cluster 0 จะได้รับผลกระทบมากสุดจาก snow และ cold และ cluster 3 rainsได้รับผลกรทบมากที่สุด 

## 6. Visualize, report, and present the problem solving steps and final solution.

อันดับแรก เราต้องสร้างแผนที่สหรัฐอเมริกาก่อน  แผนที่ที่สองก็จะแสดงสนามบินและจำนวนสภาพอากาศที่เกิดขึ้นในสนามบินนั้นและจะแสดงให้เราเห็นกลุ่มที่เราได้รับผ่าน k-Means

In [ ]:
latitude = 38.500000
longitude = -95.665

map_USA = folium.Map(location=[latitude, longitude], zoom_start=4)

map_USA

In [ ]:
airports = df[['AirportCode', 'LocationLat','LocationLng','City','State']]

airports.head()

In [ ]:
number_of_occurences = pd.DataFrame(airports['AirportCode'].value_counts())
number_of_occurences.reset_index(inplace=True)
number_of_occurences.columns = ['AirportCode', 'Count']
number_of_occurences.head()

In [ ]:
number_of_occurences = number_of_occurences.merge(airports.drop_duplicates())

number_of_occurences = number_of_occurences.merge(codes)

number_of_occurences.head()

In [ ]:
occurences = folium.map.FeatureGroup()
n_mean = number_of_occurences['Count'].mean()

for lat, lng, number, city, state in zip(number_of_occurences['LocationLat'],
                                         number_of_occurences['LocationLng'],
                                         number_of_occurences['Count'],
                                         number_of_occurences['City'],
                                         number_of_occurences['State'],):
    occurences.add_child(
        folium.vector_layers.CircleMarker(
            [lat, lng],
            radius=number/n_mean*5, # define how big you want the circle markers to be
            color='yellow',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6,
            tooltip = str(number)+','+str(city) +','+ str(state)
        )
    )

map_USA.add_child(occurences)

จะเห็นได้ว่าสนามบินที่เลือกลงจอดมากที่สุดก็จะเป็นทางเหนือของชายฝั่งตะวันตก แต่ทั่วไปที่ได้รับความเดือดร้อนน้อยสุดก็จะออกห่างจากชายฝั่งจากเหตุการณ์สภาพอากาศ

มาดู clusters ของเรา

In [ ]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=4)

# set color scheme for the clusters
x = np.arange(4)
ys = [i + x + (i*x)**2 for i in range(4)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lng, cluster, city, state in zip(number_of_occurences['LocationLat'], number_of_occurences['LocationLng'],  
                                            number_of_occurences['cluster'],
                                         number_of_occurences['City'],
                                         number_of_occurences['State']):
    #label = folium.Popup(str(city)+ ','+str(state) + '- Cluster ' + str(cluster), parse_html=True)
    folium.vector_layers.CircleMarker(
        [lat, lng],
        radius=5,
        #popup=label,
        tooltip = str(city)+ ','+str(state) + '- Cluster ' + str(cluster),
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.9).add_to(map_clusters)
       
map_clusters

 จะเห็นว่า clusters 0 ซึ่งเป็นกลุ่มที่ได้รับผลกระทบมากที่สุดจากเหตุการณ์หิมะส่วนใหญ่ตั้งอยู่ทางตอนเหนือของสหรัฐอเมริกาใกล้กับแคนาดา

สรุปบางทีการใช้ 5 กลุ่มอาจเปิดเผยให้เราดูข้อมูลของเราได้ดีขึ้น clusters 1 ยังไม่ได้กำหนดไว้อย่างชัดเจน นอกจากนี้การตรวจสอบที่ดีขึ้นของแต่ละ clusters อาจแสดงให้เราเห็นถึงความแตกต่างอื่นๆในแต่ละ clusters
